### Determing  the optimal # of hidden layers and neurons for an ANN .

This can be challenging and often requires experimentaion. Howeve are some guidelines and methods that can help you in making an informed decision :
- Start Simple " Begin with a simple Arch and gradually increase complexity if needed " 
- GridSearch/Random Search : Use grid search or random search to try diff arch. 
- Cross Validation : to evaluate performance of diff arch . 
- Heuristics and Rules of Thumb : Some heuristic and emperical rules can provide starting point, such as : 
    - The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer . 
    - Common preactice is to start with 1-2 hidden layers. 




In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
from sklearn.pipeline import Pipeline 
from scikeras.wrappers import KerasClassifier, KerasRegressor
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping 
import pickle


In [2]:
data =pd.read_csv("Churn_Modelling.csv")
data = data.drop(["RowNumber", "CustomerId","Surname"], axis=1)

label_rncoder_gender = LabelEncoder()
data["Gender"] = label_rncoder_gender.fit_transform(data["Gender"])

onehot_encoder_geo = OneHotEncoder(handle_unknown="ignore")
geo_encoded = onehot_encoder_geo.fit_transform(data[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

data = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)
X = data.drop("Exited", axis=1)
y= data["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
## Define a fuction to create the model and try diff parameters (KerasClassifier)

def create_model(neurons=32, layers= 1, optimizer ="adam"):
    model = Sequential()
    model.add(Dense(neurons, activation="relu", input_shape=(X_test.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation="relu"))

    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
#Create a Keras Classifier 

model = KerasClassifier(build_fn=create_model,neurons=32, layers= 1,verbose=1, epochs=50, batch_size=10)
# Define the grid search parameters 
param_grid = {
    "neurons": [16, 32,  64, 128],
    "layers": [1,2],
    "epochs": [50, 100] }
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=2)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
